In [2]:
import pdfplumber

In [3]:
import pandas as pd

In [4]:
pdf_path = "assets/mPassbook_SB-011001001623956_1760088095489.pdf"

In [5]:
transaction_threshold = 100000.00

In [6]:
all_transactions = []

In [7]:
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        tables = page.extract_tables()
        for table in tables:
            all_transactions.extend(table[1:])

In [9]:
df = pd.DataFrame(all_transactions, columns=['DATE', 'DESCRIPTION', 'CHEQUE NO', 'DEBIT', 'CREDIT', 'BALANCE'])

In [10]:
df

,DATE,DESCRIPTION,CHEQUE NO,DEBIT,CREDIT,BALANCE
0,28/05/2024,BY ONL UPI/CR/451521374436/P SRINIV/IOBA/PILLA...,,,"2,000.00","3,751.31"
1,29/05/2024,TO ONL UPI/DR/415075894117/INDIAN R/YESB/PAYTM...,,95.00,,"3,656.31"
2,29/05/2024,TO ATM WDL:RR NO:415009943767:C0180002-MADURAI...,,"2,000.00",,"1,656.31"
3,30/05/2024,TO ONL UPI/DR/415136201544/RADHA S/CNRB/RADHAS...,,"1,000.00",,656.31
4,30/05/2024,BY ATM CASH DEPOSIT:RR-NO:415119995033:CUB0231...,,,"4,100.00","4,756.31"
...,...,...,...,...,...,...
362,08/10/2025,BY TRANSFER: LOAN 502112025954356:00011,,,"2,95,000.00","2,96,256.45"
363,08/10/2025,TO CASH WITHDRAWAL:SELF:00011,,"2,00,000.00",,"96,256.45"
364,08/10/2025,TO ONL :XPRESS COUNTER-CASH WITHDRAW:011:: CA ...,,740.00,,"95,516.45"
365,09/10/2025,TO ONL UPI/DR/112393999621/HAASINI /UTIB/SAVIT...,,"95,000.00",,516.45


In [11]:
df['DEBIT'] = df['DEBIT'].fillna('0').replace('', '0')
df['CREDIT'] = df['CREDIT'].fillna('0').replace('', '0')

In [12]:
df

,DATE,DESCRIPTION,CHEQUE NO,DEBIT,CREDIT,BALANCE
0,28/05/2024,BY ONL UPI/CR/451521374436/P SRINIV/IOBA/PILLA...,,0,"2,000.00","3,751.31"
1,29/05/2024,TO ONL UPI/DR/415075894117/INDIAN R/YESB/PAYTM...,,95.00,0,"3,656.31"
2,29/05/2024,TO ATM WDL:RR NO:415009943767:C0180002-MADURAI...,,"2,000.00",0,"1,656.31"
3,30/05/2024,TO ONL UPI/DR/415136201544/RADHA S/CNRB/RADHAS...,,"1,000.00",0,656.31
4,30/05/2024,BY ATM CASH DEPOSIT:RR-NO:415119995033:CUB0231...,,0,"4,100.00","4,756.31"
...,...,...,...,...,...,...
362,08/10/2025,BY TRANSFER: LOAN 502112025954356:00011,,0,"2,95,000.00","2,96,256.45"
363,08/10/2025,TO CASH WITHDRAWAL:SELF:00011,,"2,00,000.00",0,"96,256.45"
364,08/10/2025,TO ONL :XPRESS COUNTER-CASH WITHDRAW:011:: CA ...,,740.00,0,"95,516.45"
365,09/10/2025,TO ONL UPI/DR/112393999621/HAASINI /UTIB/SAVIT...,,"95,000.00",0,516.45


In [13]:
df['DEBIT'] = df['DEBIT'].str.replace(',', '').astype(float)
df['CREDIT'] = df['CREDIT'].str.replace(',', '').astype(float)

In [14]:
large_transactions_df = df[(df['DEBIT'] >= transaction_threshold) | (df['CREDIT'] >= transaction_threshold)]

In [15]:

print(f"Found {len(large_transactions_df)} transactions of ₹{transaction_threshold:,.2f} or more:")
large_transactions_df

Found 5 transactions of ₹100,000.00 or more:


,DATE,DESCRIPTION,CHEQUE NO,DEBIT,CREDIT,BALANCE
199,24/01/2025,BY TRANSFER: LOAN 502112025943451:00011,,0.0,210000.00,"2,11,253.71"
200,24/01/2025,TO CASH WITHDRAWAL:SELF:00011,,190934.0,0.00,"20,319.71"
362,08/10/2025,BY TRANSFER: LOAN 502112025954356:00011,,0.0,295000.00,"2,96,256.45"
363,08/10/2025,TO CASH WITHDRAWAL:SELF:00011,,200000.0,0.00,"96,256.45"
366,TOTAL,,,1557397.4,1556162.54,516.45


In [16]:
import re

In [17]:
def create_summary(row):
    """
    Analyzes a transaction row to create a simple, readable summary.
    """
    desc = str(row['DESCRIPTION'])
    debit = row['DEBIT']
    credit = row['CREDIT']

    amount = debit if debit > 0 else credit
    
    if "ONL UPI" in desc:
        parts = desc.split('/')
        if len(parts) > 3:
            name = parts[3].strip()
            return f"{name} - {amount:,.2f}"

    if "FASTAG RECHARGE" in desc:
        return f"Fastag Recharge - {amount:,.2f}"
    if "ATM WDL" in desc:
        return f"ATM Withdrawal - {amount:,.2f}"
    if "ATM CASH DEPOSIT" in desc:
        return f"Cash Deposit - {amount:,.2f}"
    if "CREDIT INTEREST" in desc:
        return f"Credit Interest - {amount:,.2f}"
    if "MAINTENANCE CHARGES" in desc:
        return f"Maintenance Charges - {amount:,.2f}"
    if "DEBIT CARD CHARGE" in desc:
        return f"Debit Card Charge - {amount:,.2f}"
    if "TRANSFER: LOAN" in desc:
        return f"Loan Transfer - {amount:,.2f}"
    if "CASH WITHDRAWAL: SELF" in desc:
        return f"Cash Withdrawal (Self) - {amount:,.2f}"

    return f"Other: {' '.join(desc.split()[:3])} - {amount:,.2f}"


In [18]:
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        tables = page.extract_tables()
        for table in tables:
            all_transactions.extend(table[1:])

In [19]:
df = pd.DataFrame(all_transactions, columns=['DATE', 'DESCRIPTION', 'CHEQUE NO', 'DEBIT', 'CREDIT', 'BALANCE'])
df['DEBIT'] = df['DEBIT'].fillna('0').replace('', '0').str.replace(',', '').astype(float)
df['CREDIT'] = df['CREDIT'].fillna('0').replace('', '0').str.replace(',', '').astype(float)
df.dropna(subset=['DATE'], inplace=True) 

In [20]:
print("--- Simplified Transaction Summary ---\n")

df['SUMMARY'] = df.apply(create_summary, axis=1)

for index, row in df.iterrows():
    trans_type = "(Dr)" if row['DEBIT'] > 0 else "(Cr)"
    print(f"{row['DATE']} - {row['SUMMARY']} {trans_type}")

--- Simplified Transaction Summary ---

28/05/2024 - P SRINIV - 2,000.00 (Cr)
29/05/2024 - INDIAN R - 95.00 (Dr)
29/05/2024 - ATM Withdrawal - 2,000.00 (Dr)
30/05/2024 - RADHA S - 1,000.00 (Dr)
30/05/2024 - Cash Deposit - 4,100.00 (Cr)
01/06/2024 - GOOGLE I - 824.00 (Dr)
01/06/2024 - VIVEK R - 2,000.00 (Dr)
02/06/2024 - DIVYA R - 60.00 (Dr)
03/06/2024 - ABARNA K - 500.00 (Cr)
03/06/2024 - MR SATHI - 1,000.00 (Dr)
04/06/2024 - P SRINIV - 1,000.00 (Cr)
06/06/2024 - RAMANA S - 200.00 (Dr)
08/06/2024 - MS FATHI - 865.00 (Dr)
10/06/2024 - BP PETRO - 500.00 (Dr)
12/06/2024 - IOBA - 200.00 (Dr)
15/06/2024 - SUMATHY - 1,000.00 (Cr)
16/06/2024 - Maintenance Charges - 59.00 (Dr)
18/06/2024 - Cash Deposit - 2,000.00 (Cr)
24/06/2024 - P SRINIV - 9,000.00 (Cr)
25/06/2024 - ATM Withdrawal - 6,500.00 (Dr)
30/06/2024 - RADHA S - 2,500.00 (Dr)
30/06/2024 - Credit Interest - 84.00 (Cr)
01/07/2024 - GANGA S - 2,500.00 (Cr)
02/07/2024 - Cash Deposit - 3,000.00 (Cr)
04/07/2024 - MOHAMED - 1,200.00 (Dr)
05/

In [26]:
df

,DATE,DESCRIPTION,CHEQUE NO,DEBIT,CREDIT,BALANCE,SUMMARY
0,28/05/2024,BY ONL UPI/CR/451521374436/P SRINIV/IOBA/PILLA...,,0.0,2000.00,"3,751.31","P SRINIV - 2,000.00"
1,29/05/2024,TO ONL UPI/DR/415075894117/INDIAN R/YESB/PAYTM...,,95.0,0.00,"3,656.31",INDIAN R - 95.00
2,29/05/2024,TO ATM WDL:RR NO:415009943767:C0180002-MADURAI...,,2000.0,0.00,"1,656.31","ATM Withdrawal - 2,000.00"
3,30/05/2024,TO ONL UPI/DR/415136201544/RADHA S/CNRB/RADHAS...,,1000.0,0.00,656.31,"RADHA S - 1,000.00"
4,30/05/2024,BY ATM CASH DEPOSIT:RR-NO:415119995033:CUB0231...,,0.0,4100.00,"4,756.31","Cash Deposit - 4,100.00"
...,...,...,...,...,...,...,...
729,08/10/2025,BY TRANSFER: LOAN 502112025954356:00011,,0.0,295000.00,"2,96,256.45","Loan Transfer - 295,000.00"
730,08/10/2025,TO CASH WITHDRAWAL:SELF:00011,,200000.0,0.00,"96,256.45","Other: TO CASH WITHDRAWAL:SELF:00011 - 200,000.00"
731,08/10/2025,TO ONL :XPRESS COUNTER-CASH WITHDRAW:011:: CA ...,,740.0,0.00,"95,516.45",Other: TO ONL :XPRESS - 740.00
732,09/10/2025,TO ONL UPI/DR/112393999621/HAASINI /UTIB/SAVIT...,,95000.0,0.00,516.45,"HAASINI - 95,000.00"


In [29]:
def create_summary(row):
    """
    Analyzes a transaction row to create a simple, readable summary.
    """
    desc = str(row['DESCRIPTION'])
    amount = row['DEBIT'] if row['DEBIT'] > 0 else row['CREDIT']
    
    # Rule for UPI (extracts name)
    if "ONL UPI" in desc:
        parts = desc.split('/')
        if len(parts) > 3:
            return f"{parts[3].strip()} - {amount:,.2f}"
            
    # Rules for specific keywords
    keywords = {
        "FASTAG RECHARGE": "Fastag Recharge",
        "ATM WDL": "ATM Withdrawal",
        "ATM CASH DEPOSIT": "Cash Deposit",
        "CREDIT INTEREST": "Credit Interest",
        "MAINTENANCE CHARGES": "Maintenance Charges",
        "DEBIT CARD CHARGE": "Debit Card Charge",
        "TRANSFER: LOAN": "Loan Transfer",
        "CASH WITHDRAWAL: SELF": "Cash Withdrawal (Self)"
    }
    for key, value in keywords.items():
        if key in desc:
            return f"{value} - {amount:,.2f}"
            
    # Fallback for anything that doesn't match
    return f"Other: {' '.join(desc.split()[:3])} - {amount:,.2f}"

# --- Step 2: Read the PDF and create the initial DataFrame ---
pdf_path = 'assets/mPassbook_SB-011001001623956_1760088095489.pdf'
all_transactions = []
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        tables = page.extract_tables()
        for table in tables:
            all_transactions.extend(table[1:])

df = pd.DataFrame(all_transactions, columns=['DATE', 'DESCRIPTION', 'CHEQUE NO', 'DEBIT', 'CREDIT', 'BALANCE'])
df['DEBIT'] = df['DEBIT'].fillna('0').replace('', '0').str.replace(',', '').astype(float)
df['CREDIT'] = df['CREDIT'].fillna('0').replace('', '0').str.replace(',', '').astype(float)
df.dropna(subset=['DATE'], inplace=True)

# --- Step 3: Create the initial summary and a clean 'Entity' column ---
df['SUMMARY'] = df.apply(create_summary, axis=1)
# This extracts just the name/service part for grouping (e.g., "P SRINIV")
df['ENTITY'] = df['SUMMARY'].apply(lambda x: x.split(' - ')[0])

# --- Step 4: Group by 'ENTITY' and calculate totals ---
print("Aggregating data...")
# Group by the entity and sum up all their debits and credits
# This is the updated version with transaction counts
summary_df = df.groupby('ENTITY').agg(
    Total_Credit=('CREDIT', 'sum'),
    Total_Debit=('DEBIT', 'sum'),
    Credit_Entries=('CREDIT', lambda x: (x > 0).sum()), # <-- Add this line
    Debit_Entries=('DEBIT', lambda x: (x > 0).sum())   # <-- Add this line
).reset_index()

# Calculate the net amount for each entity
summary_df['Net_Amount'] = summary_df['Total_Credit'] - summary_df['Total_Debit']

# Sort the results to show the largest net transactions first
summary_df = summary_df.sort_values(by='Net_Amount', key=abs, ascending=False)

# --- Step 5: Save the final summary to an Excel file ---
output_filename = 'Transaction_Summary.xlsx'
summary_df.to_excel(output_filename, index=False)

print(f"\n✅ Success! Summary saved to '{output_filename}'")
print("\nHere is a preview of the summary:")
# Display the final summary DataFrame
summary_df

Aggregating data...

✅ Success! Summary saved to 'Transaction_Summary.xlsx'

Here is a preview of the summary:


,ENTITY,Total_Credit,Total_Debit,Credit_Entries,Debit_Entries,Net_Amount
36,Loan Transfer,545000.0,0.0,3,0,545000.0
62,Other: TO CASH WITHDRAWAL:SELF:00011,0.0,390934.0,0,2,-390934.0
6,ATM Withdrawal,0.0,370800.0,0,51,-370800.0
15,Cash Deposit,365500.0,0.0,42,0,365500.0
89,SRINIVAS,278370.0,4700.0,26,7,273670.0
...,...,...,...,...,...,...
90,SUGAM ME,0.0,65.0,0,1,-65.0
59,Other: BY ONL 0000IMPSPTM422902951147:WTB,45.0,0.0,1,0,45.0
73,Other: TO SMS CHARGES:FOR,0.0,30.0,0,1,-30.0
92,TEA RAJA,0.0,30.0,0,1,-30.0


In [6]:
# --- Step 2: Define the function to create a clean summary from the description ---
def create_summary(row):
    """
    Analyzes a transaction row with improved rule priority.
    """
    desc = str(row['DESCRIPTION'])
    amount = row['DEBIT'] if row['DEBIT'] > 0 else row['CREDIT']
    
    if "ONL UPI" in desc and "GPAYRECHAR" in desc:
        return f"Google Pay Recharge - {amount:,.2f}"

    # General keywords (made case-insensitive)
    desc_upper = desc.upper()
    keywords = {
        "FASTAG RECHARGE": "Fastag Recharge",
        "ATM WDL": "ATM Withdrawal",
        "ATM CASH DEPOSIT": "Cash Deposit",
        "CREDIT INTEREST": "Credit Interest",
        "MAINTENANCE CHARGES": "Maintenance Charges",
        "DEBIT CARD CHARGE": "Debit Card Charge",
        "TRANSFER: LOAN": "Loan Transfer",
        "CASH WITHDRAWAL: SELF": "Cash Withdrawal (Self)"
    }
    for key, value in keywords.items():
        if key in desc_upper:
            return f"{value} - {amount:,.2f}"
            
    # --- The general UPI rule now acts as a fallback ---
    if "ONL UPI" in desc:
        parts = desc.split('/')
        if len(parts) > 3:
            return f"{parts[3].strip()} - {amount:,.2f}"
            
    # Fallback for anything that doesn't match
    return f"Other Transaction - {amount:,.2f}"

# --- Step 3: Set file path, then extract and clean data from the PDF ---
try:
    pdf_path = 'assets/mPassbook_SB-011001001623956_1760088095489.pdf'
    all_transactions = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                all_transactions.extend(table[1:])

    # Create the main DataFrame
    df = pd.DataFrame(all_transactions, columns=['DATE', 'DESCRIPTION', 'CHEQUE NO', 'DEBIT', 'CREDIT', 'BALANCE'])
    
    # Clean the data
    df['DEBIT'] = df['DEBIT'].fillna('0').replace('', '0').str.replace(',', '').astype(float)
    df['CREDIT'] = df['CREDIT'].fillna('0').replace('', '0').str.replace(',', '').astype(float)
    df.dropna(subset=['DATE'], inplace=True)
    
    print("✅ PDF read and data cleaned successfully.")

except FileNotFoundError:
    print(f"❌ ERROR: The file was not found at '{pdf_path}'. Please make sure the PDF is in the same folder as the script.")
    exit()


# --- Step 4: Create a clean 'Entity' column for grouping ---
df['SUMMARY'] = df.apply(create_summary, axis=1)
# Extracts just the name/service part (e.g., "P SRINIV") for grouping
df['ENTITY'] = df['SUMMARY'].apply(lambda x: x.split(' - ')[0])


# --- Step 5: Group by 'ENTITY' and calculate totals and counts ---
print("📊 Aggregating data by entity...")
summary_df = df.groupby('ENTITY').agg(
    Total_Credit=('CREDIT', 'sum'),
    Total_Debit=('DEBIT', 'sum'),
    Credit_Entries=('CREDIT', lambda x: (x > 0).sum()),
    Debit_Entries=('DEBIT', lambda x: (x > 0).sum())
).reset_index()


# --- Step 6: Finalize the summary DataFrame ---
# Calculate the net amount for each entity
summary_df['Net_Amount'] = summary_df['Total_Credit'] - summary_df['Total_Debit']

# Sort the results to show the most significant entities first
summary_df = summary_df.sort_values(by='Net_Amount', key=abs, ascending=False)


# --- Step 7: Save the final summary to an Excel file and print a preview ---
output_filename = 'Transaction_Summary.xlsx'
summary_df.to_excel(output_filename, index=False)

print(f"\n✅ All done! Summary has been saved to '{output_filename}'")
print("\n--- Preview of Your Transaction Summary ---")
# Display the final summary DataFrame in the console
print(summary_df)

✅ PDF read and data cleaned successfully.
📊 Aggregating data by entity...

✅ All done! Summary has been saved to 'Transaction_Summary.xlsx'

--- Preview of Your Transaction Summary ---
               ENTITY  Total_Credit  Total_Debit  Credit_Entries  \
37      Loan Transfer     545000.00          0.0               3   
56  Other Transaction    1600911.08    2065648.9               7   
6      ATM Withdrawal          0.00     370800.0               0   
15       Cash Deposit     365500.00          0.0              42   
70           SRINIVAS     278370.00       4700.0              26   
..                ...           ...          ...             ...   
2              AJITHA          0.00         84.0               0   
59            PHONEPE          0.00         66.0               0   
71           SUGAM ME          0.00         65.0               0   
73           TEA RAJA          0.00         30.0               0   
79            VIJAY V          0.00          1.0               0   

In [7]:
# --- Add this block to create a total row ---
print("➕ Adding Grand Total row...")

# Create a dictionary for the new total row
total_row = {
    'ENTITY': 'GRAND TOTAL',
    'Total_Credit': summary_df['Total_Credit'].sum(),
    'Total_Debit': summary_df['Total_Debit'].sum(),
    'Credit_Entries': summary_df['Credit_Entries'].sum(),
    'Debit_Entries': summary_df['Debit_Entries'].sum(),
    'Net_Amount': summary_df['Net_Amount'].sum()
}
# Convert the dictionary to a DataFrame
total_df = pd.DataFrame([total_row])

# Append the total row to the end of your summary
summary_df = pd.concat([summary_df, total_df], ignore_index=True)

➕ Adding Grand Total row...


In [8]:
summary_df

,ENTITY,Total_Credit,Total_Debit,Credit_Entries,Debit_Entries,Net_Amount
0,Loan Transfer,545000.00,0.0,3,0,545000.00
1,Other Transaction,1600911.08,2065648.9,7,20,-464737.82
2,ATM Withdrawal,0.00,370800.0,0,51,-370800.00
3,Cash Deposit,365500.00,0.0,42,0,365500.00
4,SRINIVAS,278370.00,4700.0,26,7,273670.00
...,...,...,...,...,...,...
78,PHONEPE,0.00,66.0,0,1,-66.00
79,SUGAM ME,0.00,65.0,0,1,-65.00
80,TEA RAJA,0.00,30.0,0,1,-30.00
81,VIJAY V,0.00,1.0,0,1,-1.00


In [ ]:
def analyze_statement(pdf_path: str, output_filename: str):
    """
    Main function to read, process, and analyze a PDF bank statement,
    then export a summary to an Excel file.
    """
    print("--- Starting Bank Statement Analysis ---")

    # --- Step 2: Define the function to create a clean summary from the description ---
    def create_summary(row):
        """
        Analyzes a transaction row with improved rule priority to categorize it.
        """
        desc = str(row['DESCRIPTION'])
        amount = row['DEBIT'] if row['DEBIT'] > 0 else row['CREDIT']
        
        # Rule for Google Pay recharges (high priority)
        if "ONL UPI" in desc and "GPAYRECHAR" in desc:
            return f"Google Pay Recharge - {amount:,.2f}"

        # General keywords (made case-insensitive)
        desc_upper = desc.upper()
        keywords = {
            "FASTAG RECHARGE": "Fastag Recharge",
            "ATM WDL": "ATM Withdrawal",
            "ATM CASH DEPOSIT": "Cash Deposit",
            "CREDIT INTEREST": "Credit Interest",
            "MAINTENANCE CHARGES": "Maintenance Charges",
            "DEBIT CARD CHARGE": "Debit Card Charge",
            "TRANSFER: LOAN": "Loan Transfer",
            "CASH WITHDRAWAL: SELF": "Cash Withdrawal (Self)"
        }
        for key, value in keywords.items():
            if key in desc_upper:
                return f"{value} - {amount:,.2f}"
                
        # General UPI rule (acts as a fallback for other UPIs)
        if "ONL UPI" in desc:
            parts = desc.split('/')
            if len(parts) > 3:
                return f"{parts[3].strip()} - {amount:,.2f}"
                
        # Fallback for anything that doesn't match
        return f"Other Transaction - {amount:,.2f}"

    # --- Step 3: Extract and clean data from the PDF with error handling ---
    try:
        all_transactions = []
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                tables = page.extract_tables()
                if tables:
                    for table in tables:
                        all_transactions.extend(table[1:]) # Skip header row

        if not all_transactions:
            print("❌ ERROR: No transaction tables were found in the PDF.")
            return

        # Create and clean the main DataFrame
        df = pd.DataFrame(all_transactions, columns=['DATE', 'DESCRIPTION', 'CHEQUE NO', 'DEBIT', 'CREDIT', 'BALANCE'])
        df['DEBIT'] = df['DEBIT'].fillna('0').replace('', '0').str.replace(',', '').astype(float)
        df['CREDIT'] = df['CREDIT'].fillna('0').replace('', '0').str.replace(',', '').astype(float)
        df.dropna(subset=['DATE'], inplace=True)
        print(f"✅ PDF read successfully. Found {len(df)} total transactions.")

    except FileNotFoundError:
        print(f"❌ ERROR: The file was not found at '{pdf_path}'. Please check the file path.")
        return
    except Exception as e:
        print(f"❌ An unexpected error occurred: {e}")
        return

    # --- Step 4: Create a clean 'Entity' column for grouping ---
    df['SUMMARY'] = df.apply(create_summary, axis=1)
    df['ENTITY'] = df['SUMMARY'].apply(lambda x: x.split(' - ')[0])

    # --- Step 5: Group by 'ENTITY' and calculate totals and counts ---
    print("📊 Aggregating data by entity...")
    summary_df = df.groupby('ENTITY').agg(
        Total_Credit=('CREDIT', 'sum'),
        Total_Debit=('DEBIT', 'sum'),
        Credit_Entries=('CREDIT', lambda x: (x > 0).sum()),
        Debit_Entries=('DEBIT', lambda x: (x > 0).sum())
    ).reset_index()

    # --- Step 6: Add a 'Grand Total' row for verification ---
    print("➕ Adding Grand Total row...")
    total_row = {
        'ENTITY': 'GRAND TOTAL',
        'Total_Credit': summary_df['Total_Credit'].sum(),
        'Total_Debit': summary_df['Total_Debit'].sum(),
        'Credit_Entries': summary_df['Credit_Entries'].sum(),
        'Debit_Entries': summary_df['Debit_Entries'].sum()
    }
    total_df = pd.DataFrame([total_row])
    summary_df = pd.concat([summary_df, total_df], ignore_index=True)

    # --- Step 7: Save the final summary to an Excel file ---
    summary_df.to_excel(output_filename, index=False)
    
    print("\n🎉 --- ANALYSIS COMPLETE --- 🎉")
    print(f"✅ Summary has been saved to '{output_filename}'")
    print("\n--- Preview of Your Transaction Summary ---")
    print(summary_df)


if __name__ == '__main__':
    # --- Configuration ---
    PDF_FILE_PATH = 'assets/mPassbook_SB-011001001623956_1760088095489.pdf'
    OUTPUT_EXCEL_FILE = 'Transaction_Summary.xlsx'
    
    # --- Run the analysis ---
    analyze_statement(PDF_FILE_PATH, OUTPUT_EXCEL_FILE)

--- Starting Bank Statement Analysis ---
✅ PDF read successfully. Found 367 total transactions.
📊 Aggregating data by entity...
➕ Adding Grand Total row...

🎉 --- ANALYSIS COMPLETE --- 🎉
✅ Summary has been saved to 'Transaction_Summary.xlsx'

--- Preview of Your Transaction Summary ---
         ENTITY  Total_Credit  Total_Debit  Credit_Entries  Debit_Entries
0      ABARNA K       3020.00          0.0               4              0
1      AHASH AU          0.00       7000.0               0              1
2        AJITHA          0.00         84.0               0              1
3      AJMEER M          0.00       3524.0               0              6
4       ALBIN V          0.00       3000.0               0              1
..          ...           ...          ...             ...            ...
78     VELAVANS          0.00        570.0               0              2
79      VIJAY V          0.00          1.0               0              1
80     VIJAYAKU          0.00      15472.0     

: 